# Langfuse Python SDK

[Open in Google Colab](http://colab.research.google.com/github/langfuse/langfuse-demo-python/blob/main/notebook.ipynb)

## Install package

In [33]:
%pip install --extra-index-url https://pypi.buildwithfern.com finto-fern-langfuse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.buildwithfern.com


In [34]:
from finto.client import FintoLangfuse

In [35]:
from datetime import datetime

## Set environment

In [36]:
ENV_HOST = "https://cloud.langfuse.com"
ENV_SECRET_KEY = "sk-lf-1234567890"
ENV_PUBLISHABLE_KEY = "pk-lf-1234567890"

## Create client

In [37]:
client = FintoLangfuse(
    environment=ENV_HOST,
    password=ENV_SECRET_KEY,
    username=ENV_PUBLISHABLE_KEY
)

## Tracing

In [38]:
from finto.resources.trace.types.create_trace_request import CreateTraceRequest

trace = client.trace.create(
    request = CreateTraceRequest(
        name="chat-completion",
        metadata= {
            "key": "value"
        }
    )
)

In [39]:
from finto.resources.generations.types.create_log import CreateLog
from finto.resources.generations.types.llm_usage import LlmUsage
from finto.resources.span.types.create_span_request import CreateSpanRequest
from finto.resources.span.types.update_span_request import UpdateSpanRequest

generation = client.generations.log(
    request=CreateLog(
        traceId=trace.id,
        name="test",
        startTime=datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
        endTime=datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
        model="gpt-3.5-turbo",
        modelParameters= {
            "temperature":0.9,
            "maxTokens":1000,
            "topP":None,
        },
        prompt=[{"role": "user", "content":"Hello, how are you?"}],
        completion="I am fine, thank you",
        usage=LlmUsage(
            prompt_tokens=512,
            completion_tokens=49
        ),
        metadata= {
            "userid":'user__935d7d1d-8625-4ef4-8651-544613e7bd22',
        }
    )
)


span = client.span.create(
    request=CreateSpanRequest(
        traceId=trace.id,
        name="chat-completion",
        startTime=datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
        endTime=datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
        metadata={
            "key": "value"
        },
        parentObservationId=generation.id,
    )
)

In [40]:
from finto.resources.event.types.create_event_request import CreateEventRequest

event = client.event.create(
    request=CreateEventRequest(
        traceId=trace.id,
        name="chat-docs-retrieval",
        startTime=datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
        metadata={
            "key": "value"
        },
        parentObservationId=span.id,
    )
)

## Scoring

In [41]:
from finto.resources.score.types.create_score_request import CreateScoreRequest
score = client.score.create(
    request=CreateScoreRequest(
        traceId=trace.id,                  # trace the score is related to
        name="user-explicit-feedback",
        value=1,
        observationId=generation.id           # optionally: also attach the score to an individual observation
    )
)